## **Работа с транзакциями и сложными запросами в GORM**.

### Задачи занятия:
1. Понять, как использовать транзакции в GORM.
2. Научиться выполнять сложные запросы с использованием условий и группировок.
3. Реализовать операции с откатом в случае ошибок.

---

### Теоретическая часть:

**GORM** — это популярная библиотека ORM (Object-Relational Mapping) для языка программирования Go. Она упрощает взаимодействие с базами данных, предоставляя разработчику возможность работать с объектами и методами вместо написания SQL-запросов вручную.

### Основные возможности GORM:
1. **Модели и миграции:**
   - GORM позволяет определять структуры данных (модели) в виде Go-структур и автоматически синхронизировать их с базой данных с помощью миграций.
   - Пример модели:
     ```go
     type User struct {
         ID    uint   `gorm:"primaryKey"`
         Name  string
         Email string
     }
     ```

2. **CRUD-операции:**
   - GORM поддерживает основные операции: создание, чтение, обновление и удаление записей.
   - Пример:
     ```go
     var user User
     db.First(&user, 1) // Найти пользователя с ID 1
     ```

3. **Фильтрация и сортировка:**
   - GORM предоставляет методы для фильтрации и сортировки данных.
   - Пример:
     ```go
     db.Where("name = ?", "John").Order("created_at desc").Find(&users)
     ```

4. **Ассоциации:**
   - GORM поддерживает ассоциации между моделями, такие как `one-to-one`, `one-to-many`, `many-to-many`.
   - Пример связи:
     ```go
     type User struct {
         ID    uint
         Name  string
         Orders []Order // Связь один-ко-многим
     }

     type Order struct {
         ID     uint
         UserID uint
         Amount float64
     }
     ```

5. **Транзакции:**
   - GORM предоставляет поддержку транзакций для выполнения нескольких операций в одной последовательности.
   - Пример:
     ```go
     tx := db.Begin()
     if err := tx.Create(&user).Error; err != nil {
         tx.Rollback()
     }
     tx.Commit()
     ```

6. **Миграции и управление схемами:**
   - Автоматическая генерация таблиц и управление их структурами.
   - Пример:
     ```go
     db.AutoMigrate(&User{})
     ```

### Преимущества GORM:
- Упрощает работу с базами данных, скрывая сложность SQL.
- Позволяет описывать связи между сущностями на уровне кода.
- Поддерживает большинство реляционных баз данных, включая PostgreSQL, MySQL, SQLite и другие.

### Недостатки GORM:
- Может быть избыточным для простых проектов, где требуется высокая производительность.
- Использование абстракций GORM может добавить небольшой оверхед по сравнению с ручным SQL.

GORM — это мощный инструмент для работы с базами данных в проектах на Go, особенно если требуется удобное управление сложными схемами данных или интеграция с существующими базами.

---

### Практическая часть:

#### Шаг 1: Подготовка базы данных
1. Создайте таблицу `books` с полями:
   - `id` (Primary Key)
   - `title` (string)
   - `author` (string)
   - `year` (integer)
   - `publisher` (string, optional).

2. Наполните базу данных тестовыми данными (не менее 10 записей).

#### Шаг 2: Создание эндпоинтов для операций
1. **Эндпоинт для получения книг за указанный период**:
   - Используйте параметры `startYear` и `endYear`.
   - Реализуйте запрос с помощью метода `Where`.

   Пример:
   ```go
   func getBooksByYearRange(c *gin.Context) {
       startYear := c.Query("startYear")
       endYear := c.Query("endYear")

       var books []Book
       if err := db.Where("year BETWEEN ? AND ?", startYear, endYear).Find(&books).Error; err != nil {
           c.JSON(http.StatusInternalServerError, gin.H{"error": "Error fetching books"})
           return
       }
       c.JSON(http.StatusOK, books)
   }
   ```

2. **Эндпоинт для массового обновления издателя с транзакцией**:
   - Параметр `publisher` указывает новое значение.
   - Используйте транзакцию: при возникновении ошибки транзакция должна откатываться.

   Пример:
   ```go
   func updateBooksPublisher(c *gin.Context) {
       publisher := c.Query("publisher")
       tx := db.Begin()

       if err := tx.Model(&Book{}).Update("publisher", publisher).Error; err != nil {
           tx.Rollback()
           c.JSON(http.StatusInternalServerError, gin.H{"error": "Error updating publisher"})
           return
       }

       tx.Commit()
       c.JSON(http.StatusOK, gin.H{"message": "Publisher updated successfully"})
   }
   ```

3. **Эндпоинт для агрегатного запроса**:
   - Подсчитайте количество книг по каждому автору.
   - Используйте `Group` и `Count`.

   Пример:
   ```go
   func countBooksByAuthor(c *gin.Context) {
       var result []struct {
           Author string
           Count  int
       }

       db.Model(&Book{}).Select("author, COUNT(*) as count").Group("author").Scan(&result)
       c.JSON(http.StatusOK, result)
   }
   ```

#### Шаг 3: Тестирование
1. Проверьте корректность работы транзакций, вызвав ошибку (например, указав несуществующее поле).
2. Выполните запросы с разными диапазонами годов и проверьте результаты.
3. Убедитесь, что подсчет книг работает корректно.

---

### Задание
1. Внедрите рассмотренную выше технологию в Ваш основной проект.
2. Добавьте логирование действий в консоль для каждого этапа выполнения транзакции.

---
